In [ ]:
pip install keras-tuner

     |████████████████████████████████| 71kB 7.4MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=93103b32a206bf84645ab44bf7210bc4cf6653b8b2a0b68f0da36fa2bc56d13a
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=9ad5cf13ed187dc1264f767e5e049927d8bbfb315556e1398f75f3e96f114918
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:
import tensorflow as tf
from tensorflow import keras 
import numpy as np 
 

In [ ]:
fashion_mnist=keras.datasets.fashion_mnist 


In [ ]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()
 


4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
train_images=train_images/225.0 
test_images=test_images/225.0


In [ ]:
train_images[0].shape 


(28, 28)

In [ ]:
train_images=train_images.reshape(len(train_images),28,28,1) 
test_images=test_images.reshape(len(test_images),28,28,1) 


In [28]:
def build_model(hp):
  model=keras.Sequential([
    keras.layers.Conv2D(
      filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
      kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
      activation='relu',
      input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
      filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
      kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
      activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
      units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
      activation='relu'
    ),
    keras.layers.Dense(10,activation='softmax')#output layer
  ])
    
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2,1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

  return model
  

In [21]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [33]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

INFO:tensorflow:Reloading Oracle from existing project output/Mnist Fashion/oracle.json


In [35]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 14m 22s]
val_accuracy: 0.8728333115577698

Best val_accuracy So Far: 0.9071666598320007
Total elapsed time: 01h 25m 55s

Search: Running Trial #6

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |96                |80                
conv_1_kernel     |5                 |3                 
conv_2_filter     |32                |48                
conv_2_kernel     |5                 |5                 
dense_1_units     |64                |112               
learning_rate     |0.001             |0.001             

Epoch 1/3
 197/1688 [==>...........................] - ETA: 5:52 - loss: 0.7173 - accuracy: 0.7400

KeyboardInterrupt: ignored

In [36]:
model=tuner_search.get_best_models(num_models=1)[0]


In [37]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 80)        800       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        96048     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               2602096   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 2,700,074
Trainable params: 2,700,074
Non-trainable params: 0
_________________________________________________________________
